# Building Real MCP Servers and Clients

This notebook demonstrates building MCP servers using the official MCP SDK, then explains what happens under the hood.

What we will demonstrate is creating an MCP Server that clients can consume (internal or external).

Then we will build an MCP client that can connect to an MCP Server.


## Setup

In [ ]:
# Install required packages - using the real MCP SDK
!pip install "mcp[cli]" httpx "semantic-kernel[mcp]" python-dotenv

In [ ]:
import tempfile
import sys
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
import os


from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("✅ Setup complete!")

## Part 1: Building a Real MCP Server (Using Official SDK)

Since we are working in a Jupyter notebook, we will define the entire MCP server program as a string variable then write that string to a temporary file which becomes a runnable python script. Jupyter notebooks run in one python process, but MCP servers need to run as a separate process. 

We will use the official MCP SDK that will handle MCP protocol automatically. 

We will create an MCP server instance called weather.

Then we will create a function to call NWS API to get weather information.

mcp.tool() is the decorator where the magic happens. This registers the function as an MCP tool with the server. The FastMCP server automatically collects all decorated functions and makes them available through the MCP protocol.

We have get alerts, get forecast, and get time. 



In [ ]:
# Create a weather MCP server using the official MCP SDK
weather_server_code = '''
from typing import Any
import httpx
from mcp.server.fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP("weather")

# Constants
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"

async def make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()
        except Exception:
            return None

def format_alert(feature: dict) -> str:
    """Format an alert feature into a readable string."""
    props = feature["properties"]
    return f"""
Event: {props.get('event', 'Unknown')}
Area: {props.get('areaDesc', 'Unknown')}
Severity: {props.get('severity', 'Unknown')}
Description: {props.get('description', 'No description available')}
Instructions: {props.get('instruction', 'No specific instructions provided')}
"""

@mcp.tool()
async def get_alerts(state: str) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    url = f"{NWS_API_BASE}/alerts/active/area/{state}"
    data = await make_nws_request(url)

    if not data or "features" not in data:
        return "Unable to fetch alerts or no alerts found."

    if not data["features"]:
        return "No active alerts for this state."

    alerts = [format_alert(feature) for feature in data["features"]]
    return "\\n---\\n".join(alerts)

@mcp.tool()
async def get_forecast(latitude: float, longitude: float) -> str:
    """Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    """
    # First get the forecast grid endpoint
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    points_data = await make_nws_request(points_url)

    if not points_data:
        return "Unable to fetch forecast data for this location."

    # Get the forecast URL from the points response
    forecast_url = points_data["properties"]["forecast"]
    forecast_data = await make_nws_request(forecast_url)

    if not forecast_data:
        return "Unable to fetch detailed forecast."

    # Format the periods into a readable forecast
    periods = forecast_data["properties"]["periods"]
    forecasts = []
    for period in periods[:5]:  # Only show next 5 periods
        forecast = f"""
{period['name']}:
Temperature: {period['temperature']}°{period['temperatureUnit']}
Wind: {period['windSpeed']} {period['windDirection']}
Forecast: {period['detailedForecast']}
"""
        forecasts.append(forecast)

    return "\\n---\\n".join(forecasts)

@mcp.tool()
async def get_time() -> str:
    """Get the current time for demo purposes."""
    from datetime import datetime
    return f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

if __name__ == "__main__":
    # This single line starts the entire MCP server!
    mcp.run(transport='stdio')
'''

# Save the real MCP server to a file
with tempfile.NamedTemporaryFile(mode='w', suffix='_weather_server.py', delete=False) as f:
    f.write(weather_server_code)
    weather_server_path = f.name

print(f"✅ Created real MCP weather server at: {weather_server_path}")
print("\n🌦️  This server uses the OFFICIAL MCP SDK and provides:")
print("  • get_alerts(state) - Get weather alerts for US states")
print("  • get_forecast(latitude, longitude) - Get weather forecast")
print("  • get_time() - Get current time (for demo)")

## Part 2: Connect Semantic Kernel Agent to Real MCP Server

This creates an AI agent that can connect to our MCP weather server. We will use Semantic Kernel like before to connect to this MCP server. 

We use MCPStdioPlugin which is Semantic Kernel’s connector for MCP servers. We register it as a plugin, then provide it as a plugin to Semantic Kernel. 

What happens under the hood:

When you use the mcp.tool() decorator, under the hood, the FastMCP library registers the function, parses the name and description. This is the magic that transforms your simple python function into an MCP tool that any MCP client can discover. 

mcp.run() creates an infinite loop that constantly waits for messages from clients. 

In real world scenarios, we would connect to databases, CRM systems, and other production systems. 

In [ ]:
async def demo_real_mcp_server():
    """Connect to our real MCP server using Semantic Kernel."""
    
    if OPENAI_API_KEY == "your-openai-api-key-here":
        print("⚠️  Please set your OpenAI API key!")
        return
    
    # Create kernel with OpenAI service
    kernel = Kernel()
    chat_service = OpenAIChatCompletion(
        ai_model_id="gpt-4o-mini",
        api_key=OPENAI_API_KEY
    )
    kernel.add_service(chat_service)
    
    try:
        # Connect to our REAL MCP server
        print("🔌 Connecting to real MCP weather server...")
        
        async with MCPStdioPlugin(
            name="RealWeather",
            description="Real weather server using official MCP SDK",
            command=sys.executable,
            args=[weather_server_path],
            load_tools=True,
            load_prompts=False,
            request_timeout=30
        ) as weather_plugin:
            
            # Add MCP plugin to kernel
            kernel.add_plugin(weather_plugin)
            print("✅ Connected to real MCP server!")
            
            # Add MCP plugin to kernel
            kernel.add_plugin(weather_plugin)
            print("✅ Connected to real MCP server!")
            
            # Show that tools are available (we'll see them in action)
            print(f"\n🔧 MCP Server Connected:")
            print(f"  • Server: {weather_plugin.name}")
            print(f"  • Tools: Available via MCP protocol")
            print(f"  • Ready to use: get_alerts, get_forecast, get_time")
            
            # Create weather agent
            agent = ChatCompletionAgent(
                kernel=kernel,
                name="WeatherAgent",
                instructions="""
                You are a helpful weather assistant with access to real weather data via MCP.
                
                You can:
                - get_alerts: Get weather alerts for US states (use 2-letter codes like CA, NY, TX)
                - get_forecast: Get detailed forecasts for specific coordinates
                - get_time: Get current time
                
                Always provide helpful, accurate information using your MCP tools.
                """
            )
            
            # Test the real MCP server
            print(f"\n🌦️  Testing Real MCP Weather Server:")
            print("=" * 45)
            
            test_queries = [
                "What time is it?",
                "Are there any weather alerts in California?",
                "Get me the forecast for Sacramento (latitude 38.5816, longitude -121.4944)"
            ]
            
            for i, query in enumerate(test_queries, 1):
                print(f"\n{i}. 👤 User: {query}")
                
                try:
                    response = await agent.get_response(query)
                    print(f"   🤖 Agent: {response.content}")
                except Exception as e:
                    print(f"   ❌ Error: {e}")
            
    except Exception as e:
        print(f"❌ Real MCP connection failed: {e}")
        print("📝 Note: This requires internet access for weather API calls")

# Run the real MCP demo
await demo_real_mcp_server()